Import required libraries and list objects from the bucket:

In [3]:
import boto3
import duckdb
import pandas as pd
import numpy as np
import pyarrow
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import polars as pl


s3 = boto3.resource('s3',
  endpoint_url = 'https://7a28e49ec5f4a60c66f216392792ac38.r2.cloudflarestorage.com',
  aws_access_key_id = '37d38a72f67e6c2e6afae800b9ba8f1f',
  aws_secret_access_key = 'f5c86d7e82445f2b64bd9e260dbbc7fc2042126793149c1ab6c1d7b4672eb224'
)
bucket = s3.Bucket('all-contracts-hex')
keys = []

print('Objects:')
for item in bucket.objects.all():
#   print(' - ', item.key)
  keys.append(item.key)

start = 0
frames = [] 
dfs = []
end = len(keys) 
for i in range(start,100):
#   print(' - ', item.key)
  if keys[i].endswith('.parquet'):
    link = "SELECT block_number FROM read_parquet('https://all-contracts-eth-hex.sqd-datasets.io/"+keys[i]
    link+="');"

    #dfi = duckdb.sql(link).df()
    dfi = duckdb.sql(link).pl()

   
    dfs.append(dfi)
    #frames.append(dfi)

df = pl.concat(dfs) 
#result = pd.concat(dfs)
print(df)


Objects:
shape: (1_335_785, 1)
┌──────────────┐
│ block_number │
│ ---          │
│ u64          │
╞══════════════╡
│ 47205        │
│ 48162        │
│ 48172        │
│ 48173        │
│ …            │
│ 4160359      │
│ 4160359      │
│ 4160359      │
│ 4160359      │
└──────────────┘


Get object by key, convert to numpy dataframe and display as table:

In [6]:
g_df = df.group_by("block_number").agg(pl.col("block_number").count().alias("count"))


time_span = df.select(
    min=pl.col.block_number.first(),
    max=pl.col.block_number.last(),
).with_columns(
    
    span=pl.col.max - pl.col.min,
)


time_range = pl.arange(
    time_span[0, "min"], time_span[0, "max"], eager=True
)

grouped_df= g_df.filter(pl.col("count") != 0)
grouped_df


block_number,count
u64,u32
3381840,1
3855544,6
3760176,3
3020168,1
3943936,11
3946288,8
4036256,1
3740968,1
3956848,5


In [8]:
import plotly.express as px
import pandas as pd
import numpy as np
import datashader as ds
pandas_df1 = grouped_df.to_pandas()
cvs = ds.Canvas(plot_width=100, plot_height=100)
agg = cvs.points(pandas_df1, 'block_number', 'count')
zero_mask = agg.values == 0
agg.values = np.log2(agg.values, where=np.logical_not(zero_mask))
agg.values[zero_mask] = np.nan
fig = px.imshow(agg, origin='lower')
fig.update_traces(hoverongaps=False)
fig.update_coloraxes(showscale=False)

#fig.update_layout(coloraxis_colorbar=dict(title='Count', tickprefix='1.e'))
fig.show()

Create a plot of numbers of ERC20, ERC721 and ERC1155 deployments from block 18063600 to 1806500: